In [3]:
import requests

headers = {
    'authority': 'dce-gempak-api-prd.eco.astro.com.my',
    'accept': '*/*',
    'accept-language': 'en-MY,en;q=0.9,en-US;q=0.8,ms;q=0.7',
    'cache-control': 'no-cache',
    'origin': 'https://gempak.com',
    'pragma': 'no-cache',
    'referer': 'https://gempak.com/rojakdaily',
    'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'cross-site',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
}

params = {
    'siteId': '1',
    'pillarCategoryId': '1492',
    'language': 'en',
    'type': 'article',
    'pageSize': '100',
    'pageNumber': '3',
}

response = requests.get('https://dce-gempak-api-prd.eco.astro.com.my/v1/feed', params=params, headers=headers)

In [2]:
!mkdir url-rojakdaily

In [6]:
from tqdm import tqdm
import json
import os

def generator():
    while True:
        yield


index = 1
for _ in tqdm(generator()):
    filename = os.path.join('url-rojakdaily', f'{index}.json')
    if os.path.exists(filename):
        index += 1
        continue
        
    params = {
        'siteId': '1',
        'pillarCategoryId': '1492',
        'language': 'en',
        'type': 'article',
        'pageSize': '100',
        'pageNumber': str(index),
    }
    
    response = requests.get('https://dce-gempak-api-prd.eco.astro.com.my/v1/feed', params=params, headers=headers)
    r = response.json()
    if r['responseCode'] != 200:
        break
    
    with open(filename, 'w') as fopen:
        json.dump(r, fopen)
    
    index += 1

100it [00:14,  6.83it/s]


In [7]:
import re

def get_title(row):
    title = row['title']
    id = row['id']
    string = re.sub('[^a-z0-9 ]+', '', title.lower())
    string = re.sub(r'[ ]+', ' ', string).strip().replace(' ', '-')
    return f'{string}-{id}'

# get_title(r['response'][1])

In [8]:
from glob import glob
import json

In [9]:
files = glob('url-rojakdaily/*.json')
len(files)

100

In [10]:
urls = []
for f in files:
    with open(f) as fopen:
        data = json.load(fopen)
        
    for row in data['response']:
        urls.append('https://gempak.com/rojakdaily/entertainment/' + get_title(row))

In [11]:
len(set(urls))

10000

In [12]:
urls[0]

'https://gempak.com/rojakdaily/entertainment/usa-menang-miss-beauty-doll-2018-maya-natasha-dari-malaysia-sekadar-top-10-63660'

In [13]:
with open('url-dailyrojak.json', 'w') as fopen:
    json.dump(list(set(urls)), fopen)